This notebook re-creates the choice screens that were used in the experiment.

Each choice screen is stored as a .png file in '../results/exemplar_choice-screens/'

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Descriptives

In [ ]:
setsizes = [9, 16, 25, 36]
subjects = np.arange(49)

# stimulus image directory
data_dir = '../data/'
stimulus_dir = data_dir+'stimuli/'
# read out stimuli
stimuli = [s for s in os.listdir(stimulus_dir)]

# define output dir
output_dir = data_dir+'choice-screens/'
os.makedirs(output_dir, exist_ok=True)

# Re-create choice screens

In [ ]:
def make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir):
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    for i, stimfile in enumerate(trial_stimuli):
        # get stimulus positions
        stimbox = stimulus_positions.loc[i].values.astype(np.int)
        # get target size
        stimsize = (stimbox[2]-stimbox[0], stimbox[3]-stimbox[1])
        # load stim img
        stimimg = Image.open(stimulus_dir+stimfile)
        # resize stimulus
        stimimg = stimimg.resize(size=stimsize)
        # paste to screen
        screen.paste(stimimg, box=stimbox)
    return screen

In [ ]:
# iterate set sizes
for setsize in setsizes:
    # load data
    setsize_data = pd.read_csv(data_dir+'summary_files/{}_data.csv'.format(setsize))
    stimulus_positions = pd.read_csv(data_dir+'stimulus_positions/{}_stimulus_positions.csv'.format(setsize), header=None)
    # define stimulus centers
    stimulus_centers = np.concatenate((
        (stimulus_positions.loc[:,0].values+0.5*(stimulus_positions.loc[:,2].values-stimulus_positions.loc[:,0].values))[:,None],
        (stimulus_positions.loc[:,1].values+0.5*(stimulus_positions.loc[:,3].values-stimulus_positions.loc[:,1].values))[:,None]), axis=1)
    # iterate subjects
    for subject in subjects:
        # subset to & load subject data
        subject_data = setsize_data[setsize_data['subject']==subject].copy()
        subject_gaze_data = pd.read_csv(data_dir+'subject_files/{}_{}_fixations.csv'.format(subject, setsize))
        # extract trials
        trials = subject_gaze_data.trial.unique()
        # iterate trials
        for trial in np.unique(trials):
            # check if choice screen file exists already
            screen_filepath = output_dir+'setsize-{}_sub-{}_trial-{}.png'.format(setsize, subject, trial)
            if not os.path.isfile(screen_filepath):
                print('Processing: setsize: {}, subject: {}, trial: {}'.format(setsize, subject, trial))
                # subset data
                trial_data = subject_data[subject_data['trial']==trial].copy()
                trial_gaze_data = subject_gaze_data[subject_gaze_data['trial']==trial].copy()
                # get trial stimuli
                trial_stimuli = trial_data[['stimulus_{}'.format(i) for i in range(setsize)]].values[0]
                # make choice screen
                choice_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                # make gif & save
                choice_screen.save(screen_filepath)